In [ ]:
import pandas as pd
fp = "../data/olist_prepared/geo_SP_ED_2017.csv"
df = pd.read_csv(fp)
geo_cols = df.columns.tolist()

In [ ]:
df.values.mean()

In [ ]:
THRESHOLD=0.4
row_city = []
col_city = []
vals = []
for row in df.index:
    for col in df.index:
        if row == col:
            continue
        if df.iloc[row,col] > THRESHOLD:
            row_name = geo_cols[row]
            col_name = geo_cols[col]
            row_city.append(row_name)
            col_city.append(col_name)
            vals.append(df.iloc[row,col])

            

In [ ]:
corr_dict = {"city_1": row_city, "city_2": col_city, "corr": vals}

In [ ]:
df_corr = pd.DataFrame.from_dict(corr_dict, orient="columns")

In [ ]:
nodes = set(df_corr["city_1"]).union(set(df_corr["city_2"]))

In [ ]:
import networkx as nx
G = nx.Graph()

In [ ]:
for n in nodes:
    G.add_node(n)

In [ ]:

edge_list = tuple(zip(df_corr["city_1"], df_corr["city_2"]))

In [ ]:
G.add_edges_from(edge_list)

In [ ]:
conn_comp_size_list = [len(c) for c in sorted(nx.connected_components(G), key=len, reverse=True)]

In [ ]:
nx.number_connected_components(G)

In [ ]:
df_graph_size = pd.DataFrame({"graph_size": conn_comp_size_list})
df_graph_size["graph_size"].plot.hist()


In [ ]:
df_graph_size["graph_size"].describe()

In [ ]:
S = [G.subgraph(c).copy() for c in nx.connected_components(G)]

In [ ]:
size_indexed = {}
for c in nx.connected_components(G):
    size = G.subgraph(c).size()
    if size not in size_indexed:
        size_indexed[size] = []
    size_indexed[size].append(G.subgraph(c).copy())

In [ ]:
dict_subgraph_summ = {s: len(size_indexed[s]) for s in size_indexed.keys()}

df_subgraph_summ = pd.DataFrame.from_dict(dict_subgraph_summ, orient="index").reset_index()
df_subgraph_summ.columns = ["graph-size", "count"]

In [ ]:
df_subgraph_summ = df_subgraph_summ.sort_values(by="graph-size", ascending=False)

In [ ]:
df_subgraph_summ

In [ ]:
nx.draw(size_indexed[26][0], with_labels=True)

A review of the heatmap shows many square regions of correlation 0.4 as you move your eyes across the diagonal of the heat map. These are weeks that have high cosine similarity. I see 3 clusters for sure, can make a case for a 4 th cluster